In [1]:
from collections import deque

def find_start(graph):
    ''' returns the starting location in a 2d grid
    and makes informs if there is a start/end location
    '''
    start, end = False, False
    start_loc = None
    for row in range(len(graph)):
        for col in range(len(graph[0])):
            if graph[row][col] == 'S':
                start_loc = (row, col)
                start = True
            if graph[row][col] == 'E':
                end = True
    if not start:
        print('No Starting point found')
    if not end:
        print('No Ending point found')
    return start_loc
        

def find_shorest_path(graph):
    def explore_neighbors(current_row, current_col):
        '''Checks available paths to take,
        appends paths if constrains are satisfied'''
        row_directions = [-1, 1, 0, 0]
        col_directions = [0, 0, 1, -1]
        for row_direction, col_direction in zip(row_directions, col_directions):
            updated_row = current_row + row_direction
            updated_col = current_col + col_direction

            # boundary checks
            if updated_row < 0 or updated_col < 0:
                continue
            if updated_row > len(graph) - 1 or updated_col > len(graph[0]) - 1:
                continue

            # constraint checks
            if graph[updated_row][updated_col] == '#':
                continue

            # final check to see if the path has been explored already
            if (updated_row, updated_col) in visited:
                continue
            else:
                visited.add((updated_row, updated_col))
                path_queue.append(current_path + [(updated_row, updated_col)])

            # add to the search
            row_queue.append(updated_row)
            col_queue.append(updated_col)
    
    
    row_queue = deque()
    col_queue= deque()
    path_queue = deque()
    visited = set()
    start_row, start_col = find_start(graph)
    row_queue.append(start_row)
    col_queue.append(start_col)
    path_queue.append([(start_col, start_col)])
    while row_queue:
        current_row = row_queue.popleft()
        current_col = col_queue.popleft()
        current_path = path_queue.popleft()
        if graph[current_row][current_col] == 'E':
            return current_path
        explore_neighbors(current_row, current_col)
        
    
def print_maze(maze):
    print()
    for row in maze:
        print(row)
        
def draw_maze_path(maze, shortest_path):
    if not shortest_path:
        return 'No shortest path found'
    maze_path = maze.copy()
    for idx, path in enumerate(shortest_path):
        r, c = path
        maze_path[r][c] = str(idx)
    print_maze(maze_path)

In [8]:
''' Solve 2D maze using BFS

    problem: You are stuck in a maze and need to escape.
    You fly your drone up and draw out the map's layout.
    You want to find the shortest path to reach the exit.
    
    S = start
    E = end
    . = open path
    # = blocking wall
    
    BFS was used so solve this problem.
    There
'''
maze_2D = [
    ['.', '.', '.', '.', '.', '.'],
    ['.', 'S', '#', '#', '.', '.'],
    ['.', '#', '.', '.', '.', '.'],
    ['.', '#', '.', '#', '#', '.'],
    ['.', '#', '.', '.', 'E', '.'],
]
print_maze(maze_2D)
shortest_path = find_shorest_path(maze_2D)
draw_maze_path(maze_2D, shortest_path)


['.', '.', '.', '.', '.', '.']
['.', 'S', '#', '#', '.', '.']
['.', '#', '.', '.', '.', '.']
['.', '#', '.', '#', '#', '.']
['.', '#', '.', '.', 'E', '.']

['.', '1', '2', '3', '4', '.']
['.', '0', '#', '#', '5', '.']
['.', '#', '.', '.', '6', '7']
['.', '#', '.', '#', '#', '8']
['.', '#', '.', '.', '10', '9']


In [5]:
from collections import deque

def find_start_3D(graph):
    ''' returns the starting location in a 2d grid
    and makes informs if there is a start/end location
    '''
    start, end = False, False
    start_loc = None
    for z in range(len(graph)):
        for row in range(len(graph[0])):
            for col in range(len(graph[0][0])):
                if graph[z][row][col] == 'S':
                    start_loc = (z, row, col)
                    start = True
                if graph[z][row][col] == 'E':
                    end = True
    if not start:
        print('No Starting point found')
    if not end:
        print('No Ending point found')
    return start_loc

        

def find_shorest_path_3D(graph):
    def explore_neighbors(current_z, current_row, current_col):
        '''Checks available paths to take,
        appends paths if constrains are satisfied'''
        row_directions = [-1, 1, 0, 0, 0, 0] 
        col_directions = [0, 0, 1, -1, 0, 0]
        z_directions    = [0, 0, 0, 0, 1, -1]
        for row_direction, col_direction, z_direction in zip(row_directions, col_directions, z_directions):
            updated_row = current_row + row_direction
            updated_col = current_col + col_direction
            updated_z = current_z + z_direction
            

            # boundary checks
            if updated_row < 0 or updated_col < 0 or updated_z < 0:
                continue
            if updated_row > len(graph[0]) - 1 or updated_col > len(graph[0][0]) - 1 or updated_z > len(graph) - 1:
                continue

            # constraint checks
            if graph[updated_z][updated_row][updated_col] == '#':
                continue

            # final check to see if the path has been explored already
            if (updated_z, updated_row, updated_col) in visited:
                continue
            else:
                visited.add((updated_z, updated_row, updated_col))
                path_queue.append(current_path + [(updated_z, updated_row, updated_col)])

            # add to the search
            row_queue.append(updated_row)
            col_queue.append(updated_col)
            z_queue.append(updated_z)
    
    
    row_queue = deque()
    col_queue = deque()
    z_queue = deque()
    path_queue = deque()
    visited = set()
    start_z, start_row, start_col = find_start_3D(graph)
    row_queue.append(start_row)
    col_queue.append(start_col)
    z_queue.append(start_z)
    path_queue.append([(start_z, start_col, start_col)])
    while row_queue:
        current_row = row_queue.popleft()
        current_col = col_queue.popleft()
        current_z = z_queue.popleft()
        current_path = path_queue.popleft()
        if graph[current_z][current_row][current_col] == 'E':
            return current_path
        explore_neighbors(current_z, current_row, current_col)
        
    
def print_maze_3D(maze):
    print()
    for level, row in enumerate(maze):
        print('floor level: ', level)
        for col in row:
            print(col)
        print()
        
def draw_maze_path_3D(maze, shortest_path):
    if not shortest_path:
        return 'No shortest path found'
    print(f'\nSolution solved in {len(shortest_path)-1} steps\n')
    maze_path = maze.copy()
    for idx, path in enumerate(shortest_path):
        z, r, c = path
        maze_path[z][r][c] = str(idx)
    print_maze_3D(maze_path)

In [9]:
''' Solve 3D Maze using BFS

    problem: You are stuck in a 3D maze and need to escape.
    You fly your drone up and draw out the map's layout.
    You want to find the shortest path to reach the exit.
    
    S = start
    E = end
    . = open path
    # = blocking wall
    
    BFS was used so solve this problem.
    There
'''
maze_3D = [
    [
        ['.', '.', '.', '.', '#', '.'],
        ['.', 'S', '#', '#', '#', '.'],
        ['.', '#', '.', '.', '.', '.'],
        ['.', '#', '.', '#', '#', '#'],
        ['.', '#', '#', '.', '.', '.'],
    ],
    [
        ['.', '.', '.', '.', '.', '.'],
        ['.', '.', '#', '#', '.', '.'],
        ['.', '#', '.', '.', '#', '.'],
        ['.', '#', '.', '#', '.', '#'],
        ['.', '#', '#', '.', '.', '.'],
    ],
    [
        ['.', '.', '.', '.', '.', '.'],
        ['.', '.', '#', '#', '.', '.'],
        ['.', '#', '.', '.', '#', '.'],
        ['.', '#', '.', '#', '.', '#'],
        ['.', '#', '.', '.', 'E', '.'],
    ]
]


print_maze_3D(maze_3D)
shortest_path = find_shorest_path_3D(maze_3D)
draw_maze_path_3D(maze_3D, shortest_path)


floor level:  0
['.', '.', '.', '.', '#', '.']
['.', 'S', '#', '#', '#', '.']
['.', '#', '.', '.', '.', '.']
['.', '#', '.', '#', '#', '#']
['.', '#', '#', '.', '.', '.']

floor level:  1
['.', '.', '.', '.', '.', '.']
['.', '.', '#', '#', '.', '.']
['.', '#', '.', '.', '#', '.']
['.', '#', '.', '#', '.', '#']
['.', '#', '#', '.', '.', '.']

floor level:  2
['.', '.', '.', '.', '.', '.']
['.', '.', '#', '#', '.', '.']
['.', '#', '.', '.', '#', '.']
['.', '#', '.', '#', '.', '#']
['.', '#', '.', '.', 'E', '.']


Solution solved in 18 steps


floor level:  0
['.', '1', '2', '3', '#', '.']
['.', '0', '#', '#', '#', '.']
['.', '#', '12', '11', '10', '9']
['.', '#', '13', '#', '#', '#']
['.', '#', '#', '.', '.', '.']

floor level:  1
['.', '.', '.', '4', '5', '.']
['.', '.', '#', '#', '6', '7']
['.', '#', '.', '.', '#', '8']
['.', '#', '14', '#', '.', '#']
['.', '#', '#', '.', '.', '.']

floor level:  2
['.', '.', '.', '.', '.', '.']
['.', '.', '#', '#', '.', '.']
['.', '#', '.', '.', '#',